<a href="https://colab.research.google.com/github/jddfrance/WebscrappingTraining/blob/main/BFconcat0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Pip installs
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install lxml==4.2.5 zeep
!pip install zeep==4.0.0
!pip install --upgrade pandas --user
!pip install XlsxWriter

     |████████████████████████████████| 958 kB 4.2 MB/s 
     |████████████████████████████████| 356 kB 55.5 MB/s 
     |████████████████████████████████| 138 kB 58.0 MB/s 
     |████████████████████████████████| 3.6 MB 47.6 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

     |████████████████████████████████| 149 kB 4.2 MB/s 


In [3]:
#imports
import ssl #necessário para o MDIC, erro caso acessar sem certificados
import pandas as pd #necessário para export de xlsx do caged
import urllib.request #necessário para downloads
from urllib.request import urlopen, Request 
import requests
from requests import Session
from bs4 import BeautifulSoup 
import re 
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import os
from zeep import Client
from zeep.plugins import HistoryPlugin
from zeep import Client
from lxml import etree
from zipfile import ZipFile
import zeep
from zeep.transports import Transport
import numpy as np
from datetime import date
import xlsxwriter

In [4]:
#Download MDIC
ssl._create_default_https_context = ssl._create_unverified_context
!mkdir downloads
!mkdir raws
#link fonte: https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/balanca-comercial-brasileira-acumulado-do-ano
filenamepath = 'downloads/estado.xlsx' #NOME QUE DEVE SER UTILIZADO POSTERIORMENTE NA CONCAT
link = 'https://balanca.economia.gov.br/balanca/SH/ESTADO.xlsx'
urllib.request.urlretrieve(link, filenamepath)
df = pd.read_excel(filenamepath, sheet_name = 'DADOS_SH_UF')
df.at[0, 'Fonte/dados'] = 'gov.br Estado Produtor e Estado Importador/ Exportação e importação brasileira ' + link
dfmdicpath = 'raws/MDIC - estados.xlsx' #colocando o nome da tabela, NOME QUE DEVE SER UTILIZADO POSTERIORMENTE NA CONCAT
df.to_excel(dfmdicpath)

In [5]:
#Download Caged e destacar tabela 7
filenamepath = 'downloads/caged.xlsx'
url = 'http://pdet.mte.gov.br/novo-caged/'
response = urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')
# find all the anchor tags with "href" 
for link in soup.find_all('a', 
                          attrs={'href': re.compile(".xlsx")}): #aqui o código procura links que possuam atributo href e tenham xlsx incluido em qualquer lugar do nome
    # display the actual urls
    xlsxpath = link.get('href')
link = 'http://pdet.mte.gov.br'+xlsxpath 
urllib.request.urlretrieve(link, filenamepath)
df = pd.read_excel(filenamepath, sheet_name = 'Tabela 7')
dft7name = df.iloc[0,1]
dft7path = 'raws/caged - tabela7.xlsx' #colocando o nome da tabela, NOME QUE DEVE SER UTILIZADO POSTERIORMENTE NA CONCAT
dft7 = df.iloc[3:39,1:] #tratamento de dados simples
dft7.at[3, 'Fonte - dados'] = 'Ministério do trabalho, PDET - ' + dft7name +" "+ url
dft7.to_excel(dft7path)

In [7]:
#donwload sistema de expectativas do BCB
#refer to this link https://stackoverflow.com/questions/69875125/find-element-by-commands-are-deprecated-in-selenium/69875984
#to update the code
def get_expsys(grupoind = 'Atividade', periodo = 'Anual', indicador = 'PIB Total'):
  url = 'https://www3.bcb.gov.br/expectativas2/#/consultaSeriesEstatisticas'
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get(url)
  driver.implicitly_wait(8) #vale para todo o código e fica vigente até a identificação do id da próxima linha
  tempo = 2
  sleep(tempo)
  driver.find_element_by_id('grupoIndicador').click()
  sleep(tempo)
  driver.find_element_by_xpath(("//*[text()='" + grupoind + "']")).click()
  sleep(tempo)
  driver.find_element_by_id('periodo').click()
  sleep(tempo)
  driver.find_element_by_xpath(("//*[text()='" + periodo + "']")).click()
  sleep(tempo)
  driver.find_element_by_id('codigosIndicadores').click()
  sleep(tempo)
  driver.find_element_by_xpath(("//*[text()='" + indicador + "']")).click()
  sleep(tempo)
  dataini = driver.find_element_by_id('dataInicio')
  dataini.clear()
  sdate = datetime.today() - relativedelta(years=1)
  sdate = sdate.strftime('%d%m%Y')
  dataini.send_keys(sdate)
  sleep(tempo)
  driver.find_element_by_class_name('btn-primary').click()
  sleep(tempo+5)
  driver.find_element_by_class_name('btn-outline-secondary')
  html = driver.page_source
  #o ultimo passo é fazer um pd html para achar a tabela e transferir para o python, para então padronizar
  df = pd.read_html(html, decimal = ',', thousands = '.')
  nome = indicador +"_"+ periodo + "_"+'exp30d_mediana_ini:'+ sdate+'.xlsx'
  path = "raws/BCB - " + indicador + '.xlsx'
  df = df[0]
  df.at[0, 'Fonte - dados'] = url + " - " + indicador +" "+ periodo + " "+'expectativa dos últimos 30 dias, mediana. Data de inicio da pesquisa:'+ sdate 
  df.to_excel(path, index = False)
  driver.quit()
get_expsys()
get_expsys(grupoind = 'Índices de preços', periodo = 'Anual', indicador = 'IPCA')
get_expsys(grupoind = 'Taxas', periodo = 'Anual', indicador = 'Câmbio')
get_expsys(grupoind = 'Taxas', periodo = 'Anual', indicador = 'Selic')
get_expsys(grupoind = 'Externo', periodo = 'Anual', indicador = 'Balança comercial')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please

In [8]:
def get_sidra3653(p = 'all', v = "all"):
  clist = ['129314','129315','129316']
  loop = 0
  for ccode in clist:  
    url = 'https://apisidra.ibge.gov.br/values/t/3653/p/'+ p + '/v/'+ v + '/n1/1/n3/all/C544/'+ccode
    request=Request(url)
    response = urlopen(request)
    responseread = response.read()
    data = json.loads(responseread)
    if loop == 0:
      df = pd.json_normalize(data)
      loop = loop + 1
    else:
      df = df.append(pd.json_normalize(data))
  df.reset_index(drop = True, inplace = True)
  mes = df.D1C[df.D1C != 'Mês (Código)'].unique()
  variavel = df.D2C[df.D2C != 'Variável (Código)'].unique()
  indice = df.D4C[df.D4C != 'Seções e atividades industriais (CNAE 2.0) (Código)'].unique()
  dfrankpass = pd.DataFrame()
  for data in mes:
    for var in variavel:
      for ind in indice:
        dfrank = df[(df.NN != "Brasil") & (df.V != '...') & (df.D1C == data) & (df.D2C == var) & (df.D4C == ind)]
        dfrank["V"] = pd.to_numeric(dfrank.V, errors = 'ignore')
        dfrankpass = pd.concat([dfrankpass, dfrank.V.rank(ascending = False)])
  df['rank'] = dfrankpass
  df.at[0, 'Fonte - dados'] = url + ' - ' + 'Produção Física Industrial, por seções e atividades industriais' + ' periodo: ' + p + 'Indices: ' + v
  v = v.replace(',', '-')
  #path = 'raws/IBGE - Indústria' + p + v + '.xlsx' (levava em consideração o p e o v na nomeação)
  path = 'raws/IBGE - Indústria.xlsx'
  df.to_excel(path)

get_sidra3653(v = '3139,3140')
#get_sidra3653(p = 'last2', v = '3139,3140')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
def get_sidra(t, dados, nome, p = 'all'):
  url = 'https://apisidra.ibge.gov.br/values/t/'+ t +'/p/'+ p +'/v/all/n1/1/n3/all/C11046/90668,90669'
  request=Request(url)
  response = urlopen(request)
  responseread = response.read()
  data = json.loads(responseread)
  df = pd.json_normalize(data)
  mes = df.D1C[df.D1C != 'Mês (Código)'].unique()
  variavel = df.D2C[df.D2C != 'Variável (Código)'].unique()
  indice = df.D4C[df.D4C != 'Seções e atividades industriais (CNAE 2.0) (Código)'].unique()
  dfrankpass = pd.DataFrame()
  for data in mes:
    for var in variavel:
      for ind in indice:
        dfrank = df[(df.NN != "Brasil") & (df.V != '...') & (df.D1C == data) & (df.D2C == var) & (df.D4C == ind)]
        dfrank["V"] = pd.to_numeric(dfrank.V, errors = 'ignore')
        dfrankpass = pd.concat([dfrankpass, dfrank.V.rank(ascending = False)])
  df['rank'] = dfrankpass
  df.at[0, 'Fonte - dados'] = url + ' - ' + dados + ' periodo: ' + p
  #df.to_excel('raws/IBGE - ' + nome + " p-"+ p + '.xlsx')
  df.to_excel('raws/IBGE - ' + nome + '.xlsx')
get_sidra('3417', dados = 'Índices de volume e de receita nominal de vendas no comércio varejista ampliado', nome = 'Comércio')
#get_sidra('3417', p = 'last2', dados = 'Índices de volume e de receita nominal de vendas no comércio varejista ampliado', nome = 'Comércio')
get_sidra('6442', dados = ' Índice e variação da receita nominal e do volume de serviços', nome = 'Serviços')
#get_sidra('6442', p = 'last2', dados = ' Índice e variação da receita nominal e do volume de serviços', nome = 'Serviços')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [10]:
def get_sidraipca(dados, nome, p = 'all'):
  url = 'https://apisidra.ibge.gov.br/values/t/7060/p/'+ p +'/v/all/n1/1/n7/all/C315/7169'
  request=Request(url)
  response = urlopen(request)
  responseread = response.read()
  data = json.loads(responseread)
  df = pd.json_normalize(data)
  mes = df.D1C[df.D1C != 'Mês (Código)'].unique()
  variavel = df.D2C[df.D2C != 'Variável (Código)'].unique()
  dfrankpass = pd.DataFrame()
  for data in mes:
    for var in variavel:
      dfrank = df[(df.NN != "Brasil") & (df.V != '...') & (df.D1C == data) & (df.D2C == var)]
      dfrank["V"] = pd.to_numeric(dfrank.V, errors = 'ignore')
      dfrankpass = pd.concat([dfrankpass, dfrank.V.rank(ascending = False)])
  df['rank'] = dfrankpass
  df.at[0, 'Fonte - dados'] = url + ' - ' + dados + ' periodo: ' + p
  df.to_excel('raws/IBGE - ' + nome + '.xlsx')
  #df.to_excel('raws/IBGE - ' + nome + " p-"+ p + '.xlsx')

get_sidraipca(dados = 'Índice Nacional de Preços ao Consumidor Amplo', nome = 'IPCA')
#get_sidraipca(p = 'last2', dados = 'Índice Nacional de Preços ao Consumidor Amplo', nome = 'IPCA')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
#Download IBCR
IBCR = ['25379','25380','25383','25384','25390','25391','25392','25394','25396','25397','25398','25399','25401','25402','25404','25405','25408','25409','25410','25411','25412','25413','25415','25416','25417','25418',]
meses = pd.to_datetime(pd.date_range('2003-01-01','2021-07-01',freq = '1MS')).strftime('%d/%m/%Y')
#Preload do csv para economia de tempo de execução (16m mais veloz)
preload = pd.DataFrame([[95.38,101.41,94.09,101.73,95.28,99.52,94.02,99.72,98.91,100.48,100.38,103.37,100.87,102.6,102.03,102.02,97.33,102.04,102.34,89.08,98.58,103.01,92.39,99.97,99.57,100.8],[
97.03,102.29,111.46,103.49,92.43,99.23,94.44,100.33,97.35,101.24,97.77,106.38,111.76,104.07,105.53,103.48,112.46,94.28,102.31,92.43,99.12,106.72,89.88,99.9,94.15,100.08],[
101.42,101.43,120.56,108.82,93.17,98.95,98.15,100.03,98.9,100.27,99.64,103.37,127.77,106.21,106.8,102.77,128.61,98.83,102.93,94.19,97.88,106.97,97.09,99.36,93.39,98.66],[
100.53,100.07,117.17,104.25,95.6,99.28,95.79,98.34,99.23,100.22,100.16,102.22,118.74,103.29,107.65,101.12,108.81,99.4,102.97,99.46,98.13,104.32,107.44,99.89,94.07,99.15],[
102.66,99.3,97.67,100.22,100.85,99.1,98.2,97.6,100.01,99.64,114.24,97.87,101.66,100.09,102.49,100.09,98.66,107.71,104.48,102.57,97.02,102.15,103.66,100.28,93.7,97.99],[
104.72,98.71,97.96,102.23,100.24,100.68,97.33,98.39,98.51,99.45,103.58,101.76,93.63,97.18,100.87,100.66,95.08,113.3,109.01,98.24,100.85,101.52,107.84,101.05,92.84,98.89],[
106.7,97.5,113,101.6,105.05,99,103.42,98.46,101.35,100.19,106.02,103.17,93.38,96.23,97.44,98.06,101.35,109.4,105.8,105.7,106.24,103.61,105.62,95.31,94.85,98.48],[
100.82,99.92,102.24,102.57,105.87,99.39,104.75,98.93,101.08,99.76,102.46,102.82,91.94,95.3,98.98,98.07,106.45,110.11,106.67,105.2,102.01,105.48,109,105.98,97.02,98.06],[
99.66,100.82,97.14,102.92,104.38,99.02,103.23,99.17,100.74,99.82,102.09,103.74,93.31,97.6,100.86,99.95,105.09,107.51,106.11,109.24,104.51,103.6,100.53,102.14,103.5,98.54],[
100.81,101.02,98.21,102.91,101.63,98.42,103.49,99.4,102.61,99.96,102.37,101.62,99.42,101.39,102.05,100.07,101.14,101.5,102.54,110.96,104.32,103.88,103.3,104.37,105.99,97.84],[
99.28,102.23,93.74,103.72,97.92,99.35,101.03,100.35,100.64,100.31,100.72,102.78,101.85,102.45,105.12,101.4,98.71,99.44,101.88,112.39,105.2,104.95,93.54,98.1,104.67,97.92],[
100.73,104.66,92.38,101.02,101.27,100.05,98.64,100.78,102.17,99.69,104.87,104.78,100.86,105.19,101.92,102.01,98.4,108.54,104.3,104.35,105.8,104.08,97.23,100.53,112.23,98.88],[
97.43,103.77,100.81,108.81,96.05,101,95.53,101.67,98.93,100.57,101.4,104.34,102.71,103.25,104.63,103.46,100.15,105.27,105.79,96.87,106.81,106.45,94.03,101.81,97.65,99.05],[
98.45,104.72,114.29,108.28,93.19,101.87,95.65,102.41,96.68,100.5,98.07,105.73,109.64,104.63,105.67,104.44,113.54,99.45,105.33,92.63,102.9,108.63,92.07,102.58,93.36,98.93],[
107.23,104.78,125.88,110.36,99.05,102.61,104.43,103.91,101.03,100.85,102.99,104.62,135.89,112.38,110.41,104.98,135.63,103.92,106.76,109.18,107.41,109.97,106.6,106.57,96.62,100.15],[
106.87,106.14,122.45,110,98.34,102.88,101.47,103.8,100.45,101.2,104.48,107.86,119.56,108.39,108.17,106.12,114.66,105.06,108.66,109.04,109.78,109.52,121.63,112.76,95.69,100.65],[
108.45,105.52,108.86,112.17,104.03,103.31,103.95,104.06,101.6,101.39,121.4,104.94,102.8,104.83,104.1,105.66,102.05,112.41,109.65,114.22,109.92,106.36,112.08,109.18,96.8,101.77],[
112.95,105.7,108.87,111.81,104.11,103.82,104.34,104.69,101.65,102.39,109.82,106.32,99.54,104.76,106.06,107.14,100.09,116.9,112.2,110.62,110.8,105.51,119.33,111.08,96.25,101.92],[
115.8,105.76,123.09,111.74,110.8,105.19,110.76,105.4,103.72,102.54,108.91,106.11,101.12,105.95,106.05,108.37,106.62,114.96,111.18,109.53,111.03,109.02,119.11,107.62,97.62,101.41],[
108.49,107.25,112.66,112.44,112.92,105.95,112.51,106.14,104.66,103.16,106.5,107.23,98.17,106.31,105.14,109.2,114.05,114.25,110.88,116.97,113.19,112.82,111.09,107.91,101.88,102.89],[
105.48,107.69,108.48,113.29,111.8,107.31,111.67,108.13,103.79,103.29,105.31,107.22,95.4,106.38,103.93,109.55,112.92,112.3,110.55,118.04,114.88,112.1,106.37,109.07,108.31,104],[
105.96,107.95,107.62,113.67,108.4,106.74,109.91,107.43,105.55,103.9,107.39,108.14,102.02,107.89,107.29,109.38,105.77,107.47,109.54,119.5,116.13,111.21,105.73,108.44,111.78,104.41],[
105.08,108.18,105.53,113.93,106.67,107.75,108.22,107.51,104.02,103.7,108.19,110.41,103.49,110.09,106.81,108.95,105.12,108.6,111.12,126.88,118.97,111.61,104.04,109.06,109.82,102.75],[
105.24,109.08,105.3,113.3,110.93,109.2,106.29,108.15,107.44,104.71,111.48,111.7,107.06,114.79,106.8,109.9,107.22,118.33,112.8,115.58,118.01,112,106.73,110.01,120.82,106.29],[
101.93,109.18,105.18,114.29,101.28,106.77,101.16,108.5,102.53,104.34,107.18,111.97,101.63,108.12,104,109.31,104.21,112.14,113.13,104.86,117.06,111.45,100.55,109.31,102.64,104.74],[
101.47,108.96,121.74,117.55,97.84,107.01,100.2,108.12,98.66,103.85,101.04,111.88,105.18,107.4,100.82,109.01,115.02,105.2,115.08,105.85,116.98,111.78,97.38,109.43,98.35,105.81],[
111.76,110.49,136.99,123.64,102.88,107.08,108.28,108.58,104.98,105.44,109.54,112.36,125.93,116.06,103.66,109.61,138.29,111.19,114.83,120.32,119.26,113.94,110.87,111.57,101.81,106.05],[
112.04,110.78,134.18,121.85,104.3,107.83,106.57,108.54,105.56,105.9,110.05,111.8,113.2,112.75,102.57,109.9,118.18,112.22,115.81,122.79,121.82,113,125.6,116.08,100.55,105.36],[
112.56,109.46,112.21,114.69,109.52,108.08,108.88,109.12,107.51,107.31,128.64,110.35,103.12,108.79,104.4,109.7,107.75,117.91,115.8,130.2,124.78,112.28,115.94,112.92,101.53,106.57],[
118.96,110.34,114.1,116.66,109.27,108.2,110.34,109.63,106.53,106.61,113.04,108.9,100.77,108.77,105.97,109.83,107.74,122.37,116.52,129.97,129.4,111.09,123.23,113.95,100.77,106.25],[
120.06,110.31,127.99,117.04,114.4,109.06,113.92,109.23,107.45,106.77,110.36,107.95,99.67,105.43,105.51,108.94,107.48,118.47,115.4,119.4,122.61,111.1,126.31,115.3,102.32,106.94],[
112.34,110.16,117.59,115.62,117.18,108.69,117.3,109.93,109.59,107.43,111.65,110.88,100.65,107.38,106.83,109.05,113.7,120.72,116.42,126.25,119.89,111.43,119.96,115.94,107.11,107.65],[
108.46,110.31,110.58,115.41,113.81,108.8,113.82,109.8,108.89,108.64,110.3,112.92,95.74,104.77,103.68,107.77,112.66,118.03,116.48,123.83,121.05,111.99,112.05,114.38,111.46,106.57],[
109.3,111.21,110.38,116.1,111.97,110.1,112.94,110.32,110.03,108.41,113.22,114.88,99.85,108.18,105.05,110.09,106.41,115.36,117.69,129.07,125.32,111.59,110.81,113.31,114.6,106.88],[
107.85,111.06,109.3,116.7,109.75,110.91,111.92,111.26,109.43,109.2,112.04,114.45,102.82,112.03,106.18,110.88,104.6,115.57,118.04,127.68,120.1,110.83,108.2,113.29,115.65,108.26],[
107.58,111.05,109.94,117.89,113.8,111.94,110.37,111.86,112.21,109.05,114.22,114.59,106.06,115.07,105.96,110.57,107.23,125.83,119.62,117.28,118.19,110.82,112.64,115.25,123.39,108.38],[
106.04,112.73,110.36,119.08,108.34,113.24,105.64,112.49,108.59,109.98,112.64,117.34,106.48,111.23,108.01,111.5,108.07,119.97,120.81,110.66,120.63,114.24,108.31,116.42,107.07,108.57],[
105.6,113.34,119.91,117.53,104.34,113.15,104.9,112.8,105.14,110.58,104.94,115.9,113.24,110.21,108.29,111.39,115.42,112.12,122.27,116.98,127.92,112.16,105.16,117.63,101.63,109.06],[
115.16,113.41,130.68,116.84,111.64,114.96,114.08,113.77,110.03,109.87,114.63,117.47,131.97,118.79,108.57,111.25,133.26,119.7,122.72,128.25,127.1,110.55,115.21,115.25,105.41,109.09],[
113.23,114.23,126.76,118.98,110.08,115.82,110.33,114.28,108.99,110.62,113.7,117.74,118.07,111.03,109.14,111.21,116.03,117.97,123.79,121.81,124.86,112.86,123.5,115.94,104.07,110.72],[
119.36,115.36,119.34,120.19,121.6,118.82,115.42,114.77,111.69,110.9,139.36,117.3,108.35,112.94,108.59,112.21,113.33,126.81,124.33,132.01,124.47,116.02,119.85,116.02,105.19,109.84],[
124.86,116.2,117.8,120.38,119.14,118.63,114.29,114.24,108.01,108.64,124.94,120.83,102.54,109.9,108.12,111.91,110.98,129.76,124.22,120.13,121.78,115.41,124.46,115.95,104.48,110.48],[
128.25,117.73,132.96,120.83,125.82,120.46,120.38,115.45,112.19,111.37,122.28,120.06,102.99,109.75,109.36,113.8,112.25,129.03,125.92,120.92,124.41,115.69,126.93,116.32,106.85,111.94],[
119.15,116.7,125.32,121.74,128.62,119.78,123.17,115.29,113.6,111.64,117.66,116.16,103.65,112.07,109.32,113.4,118.43,132.33,127.5,128.28,121.33,115.58,121.8,117.73,111.39,111.84],[
114.95,117.08,117.65,122.55,124.17,118.98,118.98,115.27,111.75,111.59,119.26,120.97,101.08,109.52,110.45,113.48,111.64,130.03,128.41,127.9,124,112.45,115.94,118.84,117.68,112.74],[
115.7,117.37,118.35,123.21,122.87,120.05,119.28,116.47,114.23,112.59,120.63,121.84,105.07,113.04,109.93,114.18,111.18,127.93,129.9,128.29,123.09,116.39,116.71,119.13,122.48,114.04],[
114.95,118.45,116.8,123.65,117.02,118.57,117.41,116.79,113.14,112.96,120.1,123.14,106.45,116.07,110.02,114.89,111.56,128.12,130.7,132.67,125.43,117.89,114.04,119.21,121.75,114.07],[
113.91,119.58,115.89,125.28,118.91,118.82,114.21,117.81,115.71,113.8,121.39,123.71,110.91,118.59,112.4,116.32,114.11,136.21,130.64,120.76,124.41,120.17,115.39,119.67,128.73,114.61],[
112.55,119.94,117.3,125.74,114.15,119.11,110.79,118.18,112.24,113.67,117.47,121.89,112.06,116.26,113.73,116.33,114.2,129.74,130.66,117.32,125.99,120.56,112.41,120.59,112.18,113.83],[
111.21,120.04,121.76,120.79,110.22,119.86,110.72,119.44,107.73,113.67,111.1,123.4,119.26,117.79,114.8,119.82,122.2,119.15,130.4,114.62,126.18,119.76,107.19,120,106.29,114.16],[
122.35,120.49,133.44,120.69,116.1,119.25,120.38,119.74,115.26,114.8,121.28,124.12,136.47,124.83,112.87,117.2,142.78,127.44,130.58,130.71,128.56,119.4,118.19,117.68,110.39,113.78],[
120.55,121.12,129.69,123,114.48,119.5,117.52,121.14,113.42,114.73,119.6,124.31,126.13,120.2,115.01,118.85,125.91,124.66,130.84,127.88,133.24,122.29,125.65,117.66,109.22,115.78],[
126.26,122.09,125.48,126.02,122.23,119.9,122.64,121.89,115.65,114.49,154.54,129.79,115.98,120.89,116.42,119.85,120.79,133.56,131.2,134.83,128.36,122.7,125.04,121.06,111.68,116.69],[
131.26,122.13,122.74,125.53,119.84,119.56,123.09,123.34,113.85,114.64,133.15,128.01,109.88,117.7,116.37,120.67,119.73,135.48,129.86,128.86,130.46,124.63,130.88,122.4,110.64,117.11],[
133.79,122.37,139.86,125.64,125.35,119.7,130.27,124.55,115.58,114.84,130.02,127.63,110.92,116.99,116.88,120.43,121.81,135.04,131.85,126.73,129.95,124.82,134.52,123.72,111.71,116.86],[
126.83,123.9,133.4,128.59,128.09,119.64,134.13,125.49,117.42,115.26,131.06,128.53,111.56,120.44,117.22,121.44,128.49,137.99,132.25,139.94,132.89,125.44,126.57,122.6,116.68,117.07],[
120.9,124.1,124.98,130.16,126.17,122.35,130.09,127.04,114.64,115.17,120.55,123.63,106.91,116.31,117.82,122.28,122.21,135.68,134.79,134.52,131.74,124.36,119.72,123.62,121.96,117.65],[
124.23,125.11,127.11,130.97,132.35,128.39,131.74,127.76,118.08,115.87,128.77,129.28,114.88,123.13,119.28,123.12,125.44,133.85,134.59,141.43,132.89,129.27,123.72,125.41,128.78,119.55],[
123.32,127.22,124.64,132.08,122.49,123.99,128.84,128.26,116.73,116.56,128.8,132.71,116.5,125.02,120.45,123.83,121.58,132.97,135.4,140.29,133.02,128.1,119.94,125.1,127.52,119.64],[
121.64,127.85,122.14,132.74,125.58,125.89,123.83,128.1,119.96,117.45,133.24,137.85,118.61,124.41,120.87,122.96,123.47,142.24,135.86,129.93,135.05,129.67,122.74,127.26,136.56,122.36],[
120.06,128.25,123.64,132.56,118.69,124.08,120.83,128.89,116.85,118.09,127.11,132.39,117.68,121.63,119.67,121.68,124.67,134.92,135.59,129.03,138.01,131.04,117.98,126.24,119.61,121.46],[
119.71,127.32,134.53,132.52,117.12,125.02,120.94,128.26,113.15,117.33,123.06,132.12,125.47,124.32,120.75,122.97,133.45,127.37,134.44,126.68,138.12,128.3,115.47,126.8,115.45,121.05],[
128.07,128.4,143.42,133.71,121.86,127.1,128.86,130.53,118.59,119,131.69,136.85,139.54,127.92,117.84,123.43,151.3,131.33,136.4,135.4,136.79,130.39,125.17,126.39,117.11,122.1],[
129.2,128.4,143.36,133.59,123.35,126.44,129.78,131.9,117.79,118.76,135.09,136.78,132.62,128.58,118.92,124.46,137.74,130.66,137.12,133.57,134.64,131.63,138,127.47,115.64,121.22],[
133.92,130.54,134.5,136.2,130.83,129.68,131.63,132.12,121.03,120.44,167.93,141.27,118.9,124.02,121.57,124.95,130.33,137.19,136.38,141.29,137.28,134.63,131.13,127.95,116.28,122.03],[
141.65,131.35,135.65,138.7,131.15,130.81,133.67,133.65,120.71,121.32,143.55,138.84,116.81,123.06,122.95,125.79,130.18,143.47,136.73,136.01,138.42,134.43,134.88,126.17,115.7,122.23],[
145.3,132.03,156.61,138.65,137.95,131.05,141.4,134.12,123.01,121.93,142.87,138.68,117.44,125.05,122.56,126.93,135.39,142.62,137.98,135.74,136.33,136.37,138.74,127.4,117.9,122.85],[
133.28,131.12,141.78,137.51,140.12,132.68,141.88,134.01,123.77,122.4,139.3,137.1,115.64,124.64,122.99,127.89,136.86,145.58,140.42,142.89,137.58,135.64,134.49,131.55,122.63,123.82],[
129.82,130.46,132.84,136.92,138.04,131.08,141.54,135.56,123.51,122.61,135.55,137.6,116.57,125.92,125.4,128.97,137.03,142.88,140.44,147.6,140.49,137.07,127.76,129.79,130.62,124.13],[
129.69,129.23,133.47,137.25,138.82,133.55,139.49,133.93,125.06,122.18,133.02,132.45,119.07,127.3,121.9,125,133.21,140.85,139.92,147.65,137.39,135.21,125.85,126.4,133.43,123.14],[
121.35,125.29,126.74,136.55,124.61,125.98,133.62,133.06,121.75,121.45,120.19,124.37,116.34,123.66,119.99,122.57,127.27,135.91,138.29,139.87,132.93,133.76,121.46,126.46,130.61,122.78],[
115.32,120.2,125.67,136.56,128.85,127.02,126.04,129.39,123.33,120.4,119.82,122.89,116.3,123.23,116.96,120.13,122.84,143.69,136.74,132.49,134.11,127.97,121.84,124.95,137.87,123.03],[
111.58,119.43,125.58,136.9,121.04,126.57,122.09,130.4,120.83,122.14,113.12,117.76,114.42,122.08,117.22,122.82,122.93,136.12,137.26,121.89,129.39,129.92,115.19,122.97,122.61,124.37],[
113.08,122.3,139.06,138.71,118.36,128.1,121.35,130.69,116.38,122.31,111.4,123.1,122.4,121.66,118.89,123.76,131.79,126.19,138.87,119.09,130.8,129.79,114.87,128.28,115.6,123.44],[
125.07,124.11,151.97,139.5,125.06,128.18,131.06,131.21,125.24,124.94,117.96,121.29,147.71,129.98,122.11,122.94,151.38,133.67,138.61,130.49,130.23,129.08,126.34,125.94,120.15,123.77],[
123.47,124.02,148.22,139.42,124.11,128.9,128.24,131.54,122.82,124.09,121.58,124.62,135.36,126.21,122.68,123.58,134.39,129.33,136.45,124.35,129.25,129.6,133.36,124.31,118.24,125.04],[
127,124,134.53,138.11,129.49,128.98,131.72,132.21,125.86,125.31,150.83,127.66,118.49,123.73,121.7,124.85,125.91,134.7,134.17,138.15,134.99,129.39,128.67,125.37,118.93,124.79],[
135.54,125.54,135.66,137.94,131.21,130.44,132.97,133.07,124.49,125.39,132.9,128.21,116.15,124.27,122.32,127.27,124.99,145.87,138.61,132.01,133.71,129.34,135.6,127,119.88,126.3],[
138.09,125.28,158.78,138.58,138.32,131.66,138.39,131.27,126.83,125.63,133.83,129.88,116.84,124.75,121.99,126.23,128.49,144.99,139.17,134.61,135.7,129.02,135.07,124.8,121.1,126.21],[
129.56,127.31,144.11,139.1,139.49,132.81,140.27,132.59,127.5,125.98,135.36,133.72,115.68,123.78,123.21,127.49,132.79,144.37,138.96,144.32,139.18,131.57,134.77,132.27,127.31,128.79],[
127.54,129.21,136.57,140.45,139.9,133.8,138.92,134.09,127.9,128.02,134.19,135.98,115.39,125.24,124.8,129.12,129.25,141.15,139.58,146.88,139.8,130.87,129.99,133.28,135.15,129.23],[
130.02,130.54,135.07,139.97,138.67,134.95,139.47,135.13,129.78,127.95,136.72,137.07,120.43,130.49,124.56,130.18,131.32,140.74,140.97,152.57,143.95,134.83,130.29,131.71,138.15,128.48],[
126.75,130.94,131.83,142.32,133.56,134.77,136.24,135.69,127.55,127.13,133.28,138.16,121.14,131.2,124.48,130.09,129.27,137.67,140.19,151.57,144.04,135.64,128.19,133.17,138.25,130.32],[
126.68,132.39,131,142.79,138.64,136.96,133.7,137.03,131.24,128.46,138.88,142.65,127.67,134.41,128.41,131.02,134.17,147.41,139.98,140.24,141.43,139.03,132.5,135.4,145.82,130.84],[
123.56,133.58,133.14,146.91,131.84,138.84,127.82,137.64,127.69,129.63,139.66,146.39,122.7,129.87,127.16,132.29,131.2,142.19,144.31,139.89,149.3,140.18,125.78,134.91,128.28,130.81],[
124.99,135.3,150.02,149.97,128.78,138.98,128.63,138.45,124.83,130.76,133.64,146.85,130.77,130.39,127.39,132.54,145.34,133.81,147.84,134.82,148.05,143.22,121.65,135.51,125.86,133.95],[
138.57,136.62,165.29,151.7,139.18,141.12,140.37,139.52,131.77,130.65,146.14,148.17,158.48,143.92,129.47,134.39,176.63,143.21,148.49,158.08,155.62,148.64,139.25,137.65,132.65,135.66],[
136.81,137.22,159.99,150.63,135.99,140.85,136.38,139.53,129.08,130.81,143.51,147.66,144.4,138.19,130.05,134.89,149.58,139.49,148.48,146.37,153.21,144.32,149.93,139.27,127.64,134.3],[
140.41,136.82,143.61,147.24,140.05,139.44,139.68,139.94,132.32,131.45,169.74,145.09,126.37,133.46,129.71,134.15,140.67,147.83,146.91,153.98,151.88,144.3,142.11,138.17,129.7,135.59],[
146.99,136.27,140.6,142.55,139.9,138.97,140.02,140.28,130.68,131.98,156.86,150.51,124.07,131.25,130.82,134.21,139.8,155.89,146.74,150.22,151.38,144.81,144.36,135.69,128.53,135.25],[
150.13,136.21,168.47,146.16,146.25,139.99,147.74,140.44,133.4,132.52,156.5,152.66,126.12,129.9,132.48,132.41,143.63,153.03,146.56,149.86,150.89,144.21,146.89,136.12,130.67,136.19],[
139.98,137.08,155.52,149.05,147.14,139.87,150.77,142.09,134.65,133.23,155.99,154.61,124.35,130.57,131.29,133.6,145.12,153.34,147.4,156.75,150.79,143.3,140.82,138.16,135.83,137.11],[
136.76,138.53,145.59,149.7,147.17,141.63,146.78,141.65,134.02,133.49,151.82,154.93,120.26,129.74,130.3,134,144.15,149.44,147.23,155.82,149.72,145.82,134.57,138.33,143.26,137.58],[
137.28,138.58,146.15,152.38,144.5,141.8,144.83,141.78,135.58,133.96,155.25,156.69,124.35,133.16,130.16,134.66,139.81,146.38,147.2,156.46,151.06,145.73,138.25,141.21,148.3,139.29],[
135.1,139.59,142.11,152.55,142.45,143.5,143.26,142.69,135.72,135.13,152.51,157.87,130.59,137.68,133.68,136.49,142,148.3,151.48,162.62,154.7,148.9,135.73,140.78,148.09,140.02],[
134.99,140.64,140.97,153.79,146.3,143.76,139.55,142.44,138.05,134.77,153.77,157.07,132.4,142.7,132.72,137.88,142.92,160.02,150.44,148.95,150.98,148.44,137.16,139.65,156.09,140.25],[
130.72,140.95,138.15,152.86,136.89,143.27,134.15,144.1,133.09,134.52,155.56,163.38,129.83,136.87,134.07,138.76,140.83,153.41,155.36,146.93,156.76,150.27,130.48,139.32,136.36,138.65],[
133.82,142.76,157.4,155.85,137.16,145.11,136.48,144.6,131.13,135.54,152.48,164.64,140.86,137.93,134.96,137.46,154.04,136.48,150.21,146.46,155.63,148.64,126.69,138.54,134.19,140.69],[
142.04,142.4,167.85,156.43,142.04,146.33,143.98,145.1,133.91,134.23,161.67,165.44,166.8,145.3,137.46,137.62,174.81,141.29,148.63,153.26,155.97,148.55,139.62,139.74,136.76,141.07],[
140.26,141.26,164.37,155.28,141.68,146.7,139.1,143.11,133.72,135.71,161.84,164.81,150.45,137.95,136.69,135.33,154.4,141.72,151.7,152.64,158.58,150.55,149.94,139.85,134.15,141.19],[
146.07,141.3,153.93,157.02,154.74,152.89,145.96,144.97,135.7,134.59,192.41,164.98,133.7,134.84,136.47,134.28,148.89,155.05,153.51,162.91,158.53,151.43,145.87,140.56,136.36,141.18],[
152.48,141.48,156.13,158.19,151.38,150.91,144.86,145.11,132.95,134.21,169.41,163.28,128.74,132.23,136.1,135.24,147.34,166.96,155.36,155.91,158.33,152.34,150.04,141.68,136.02,143.14],[
154.22,140.68,182.94,159.2,156.36,150.94,151.83,145.8,134.49,134.32,165.99,162.96,129.03,131.48,136.88,135.56,151.09,164.75,157.82,159.07,161.4,153.49,151.09,141.35,137.14,143.52],[
144.79,140.74,164.77,156.28,159.75,150.98,155.95,145.87,136.84,134.75,163.24,160.71,130.16,134.6,136.4,136.2,157.22,164.08,156.57,169.07,160.04,153.69,144.58,141.33,142.81,143.6],[
139.27,140.69,153.72,158.76,156.34,150.47,151.42,145.74,135.61,135.41,159.2,163.48,126.63,132.68,136.36,137.06,153.14,157.2,155.85,169.02,162.54,155.05,137.18,140.47,151.1,145.05],[
140.1,141.15,149.7,156.82,149.45,146.73,148.42,145.3,137.03,135.53,158.37,160.65,130.9,134.38,137.21,136.69,147.99,152.64,154.54,170.76,165.99,154.73,138.21,141.15,154.74,145.66],[
139.76,144.4,149.15,160.02,146.26,147.1,146.27,145.72,136.79,136.12,161.15,165.96,134.49,139.22,137.43,137.93,150.89,153.7,157.47,167.18,159.32,158.23,135.18,139.97,153.4,145.44],[
137.31,142.56,147.83,161.59,150.44,147.62,143.27,145.36,139.03,135.35,164.38,166.25,138.98,143.95,140.7,139.86,155.75,170.57,159.79,158.22,157.74,161.38,137.91,139.29,162.23,145.79],[
128.18,137.02,142.26,151.83,141.7,147.52,135.68,144.42,133.42,134.22,156.08,162.67,131.07,133.28,134.27,133.79,145.51,150.33,151.75,150.82,157.6,153.68,133.2,140.6,144.72,145.82],[
133.58,142.77,158.02,152.45,139.64,148.79,137.7,145.93,133.3,136.58,154.58,162.94,134.95,137.97,130.65,136.5,157.69,140.6,150.96,145.91,157.49,151.38,130.3,141.86,141.54,146.97],[
145.72,144.06,173.03,154.1,145.27,148.87,146.93,145.95,138.1,136.91,164.31,166.13,153.16,145.17,124.28,135.24,177.41,148.52,155.78,156.66,156.02,147.23,142.52,140.46,144.41,146.98],[
143.6,145.32,173.08,157.41,141.7,148.53,141.5,146.24,135.04,137.25,157.28,162.42,140.95,138.08,127.26,135.21,157.06,148.09,159.29,148.37,156.94,153.34,151.14,141.51,139.89,147.47],[
151.65,146.97,166.23,160.72,148.73,148.5,148.05,147.03,138.41,137.58,196.31,168.9,135.24,139.6,138.27,138.56,154.09,163.36,160.96,160.28,156.42,156.58,147.48,142.51,143.47,148.43],[
165.01,153.23,163.8,160.72,148.56,149.63,147.11,147.47,136.24,137.8,174.81,168.79,130.44,135.36,139.16,139.37,152.47,173.39,160.26,157.32,160.5,158.95,151.37,143.68,142.48,149.59],[
168.11,152.9,196.42,162.18,153.9,149.44,156.22,150,138.49,138.17,169.02,167.18,131.44,136.14,138.9,140.22,156.82,167.7,159.93,150.68,151.94,158.83,152.44,142.09,142.26,148.37],[
151.61,147.24,179.75,161.93,156.64,149.55,161.75,150.98,140.4,138.38,167.13,164.77,135.94,140.2,142.05,140.97,164.27,169.61,161.23,172.56,164.21,160.08,146.06,143.06,148.18,149.06],[
142,145.36,159.03,159.38,153.12,150.54,153.4,149.82,135.8,136.58,156.35,160.67,127.68,132.4,140.17,139.85,153.39,160.28,160.34,168.22,161.94,157.91,138.87,143.93,150.82,146.89],[
147.61,147.51,162.63,162.47,156.41,152.76,155.83,151.46,140.34,138.1,169.71,169.62,135.57,140.31,140.62,140.57,153.67,160.68,162.36,165.97,158.15,159.33,143.4,145.86,159.5,149.73],[
142.72,147.45,154.56,161.09,153.51,155.35,151.11,150.6,138.73,138.03,156.97,160.55,137.1,141.89,140.21,140.48,151.43,155.83,160.21,169.75,162.48,158.88,141,145.78,158.1,150.26],[
138.68,146.34,150.2,161.37,152.64,153.43,146.33,151.03,142.29,139.32,160.92,164.55,135.45,139.24,140.89,139.16,149.34,171.96,162.27,159.93,163.84,158.81,144.33,148.09,164.96,150.78],[
137.39,146.96,155.46,164.55,148.98,155.7,141.61,150.59,138.78,139.45,157.95,163.98,137.92,140.49,141.27,140.5,152.71,161.54,162.45,154.97,161.57,160.4,140.05,147.51,151.43,152.45],[
134.1,145.52,166.5,164.67,141.55,153.9,139.96,150.37,131.66,137.04,153.42,167.5,147.69,138.21,142.46,140.49,166.89,143.49,160.83,149.26,162.72,163.41,134.38,148.24,143.23,151.6],[
146.52,147.36,181.7,166.53,147.09,154.01,149.51,151.29,138.83,138.88,158.07,162.08,169.38,145.47,139.55,138.7,194.34,149.08,158.79,161.35,164.73,164.14,146.27,146.09,147.7,151.87],[
149.31,147.74,184.44,166.13,150.1,154.69,150.92,152.54,138.31,138.9,162.54,164.27,163.03,149.19,143.71,141.78,173.21,145.73,155.2,162.74,164.55,164.37,157.5,144.03,147.04,152.04],[
154.33,150.48,169.59,164.55,154.47,156.36,154.06,154.04,138.91,139.06,186.38,162.42,141.68,142.12,146.17,142.53,159.1,157.65,156.15,167.24,166.81,164.3,150.7,146.88,147.28,152.58],[
163.98,152.58,167.42,165.97,151.6,154.32,153.17,153.36,137.54,139.11,167.3,162.4,136.42,138.77,145.84,143.17,155.55,176.06,161.43,161.42,164.66,161.91,152.23,145.52,144.06,151.22],[
167.95,151.56,206.21,169.07,159.53,154.76,161.12,153.7,140.65,139.5,165.57,162.08,140.77,141.75,147.89,144.44,163.98,175.57,165.32,169,166.78,164.32,158.49,146.8,146.74,151.93],[
153.98,149.66,186.78,167.62,159.82,154.37,165.38,154.59,141.62,139.8,164.05,162.66,141.01,142.75,148.23,144.5,170.22,171.75,163.93,175.88,167.87,166.55,149.5,146.71,151.1,152.1],[
146.52,149.12,168.59,168.35,159.05,156.42,159.92,155.64,140.78,141.15,157.99,162.38,136.78,138.44,148.23,144.34,161.59,165.37,164.33,174.92,169.05,166,143.41,148.17,154.03,149.97],[
150,149.37,168.45,167.21,161.73,158.35,159.03,153.92,143.57,141.41,166.25,165.26,144.86,146.99,149.05,145.41,160.61,166,167.91,176.79,169.54,165.92,144.2,146.41,161.62,151.47],[
145.5,150.38,162.05,168.68,157.3,160.01,155.43,155.02,141.62,140.99,164.93,167.88,145.2,146.98,149.19,145.43,158.93,162.05,167.2,177.28,170.53,166.79,143.61,148.03,159.51,151.86],[
139.52,146.84,161.48,171.91,158.45,159,149.28,153.3,143.42,140.43,159.71,163.73,140.7,145.15,145.43,144.68,155.22,179.71,170.67,169.58,172.19,164.81,145.47,148.31,167.97,153.31],[
141.97,151.45,157.39,167.18,152.27,159.69,144.85,153.59,140.31,140.89,160.07,166.58,142.89,145.44,146.2,145.27,158.85,172.02,172.98,163.72,171.85,166.71,142.3,149.4,156.34,156.56],[
141.5,151.3,173.09,169.55,148.91,160.03,145.21,153.57,137.61,141.85,151.29,163.18,155.66,146.73,146.81,146.53,174.23,146.68,163.81,161.83,170.16,166.95,137.48,149.24,148.23,155.41],[
148.29,150.28,183.85,169.8,150.79,160.68,149.87,153.06,141.37,141.71,158.63,164.91,177.41,151.66,146.74,146.17,193.5,154.33,165.07,166.28,173.51,164.02,148.42,148.94,149.73,154.82],[
148.04,148.96,189.03,171.45,152.43,160.81,148.52,152.75,138.89,141.53,164.36,167.16,163.43,150.29,145.41,144.85,169.04,155.81,167.97,162.73,167.21,163.16,160.93,148.88,148.1,154.83],[
151.84,147.3,176.92,171.5,158.78,161.51,153.23,152.3,141.03,140.82,190.7,165.15,140.73,144.75,145.18,144.83,158.64,172.02,170.05,168.88,166.33,163.11,153.65,149.44,149.6,154.26],[
155.04,145,173.36,171.06,152.88,157.49,149.87,150.88,139.92,141.71,170.82,167.46,130.67,135.11,140.71,140.44,149.24,184.58,168.35,153.45,159.19,157.65,153.81,149.07,143.27,150.79],[
157.76,142.51,211.99,173.83,165.02,161.3,157.25,149.7,142.74,141.8,174.34,170.32,134.98,140.58,141.65,142.74,158.63,181.92,169.94,167.16,164.12,158.29,163.38,151.28,146.03,150.74],[
151.31,148.23,192.61,173.41,166.73,163.26,159.41,150.56,142.55,141.44,173.64,173.31,135.88,139.54,144.42,142.85,164.03,176.25,169.53,168.97,163.27,163.14,151.18,149.79,150.25,151.95],[
147.68,148.3,173.43,172.05,167.75,164.03,155.93,149.69,141.33,140.79,171.79,173.23,136.16,140.47,146.11,144.42,162.8,172.01,169.41,173.93,162.4,164.69,146.18,148.71,156.74,151.33],[
149.5,147.85,173.77,172.6,164.08,161.33,156.54,150.57,143.85,140.89,176.26,173.3,140.77,146.8,145.12,144.08,159.67,169.19,170.76,170.93,163.15,163.95,147.4,149.08,161.26,150.58],[
142.49,147.39,163.3,171.59,158.18,162.02,150.02,149.78,141.55,140.94,173.33,176.07,140.43,145.66,145.23,144.12,157.02,166.5,172.32,167.66,162,164.67,144.69,148.84,159.33,151.73],[
140.41,146.86,157.8,168.63,162.31,162,147.08,150.19,145.1,141,171.92,173.51,141.23,146.2,145.49,144.41,157.59,188.32,178.65,166.51,164.72,166.1,146.52,148.73,166.22,151.51],[
138.56,148.06,154.05,170.55,156.74,161.01,141.96,150.86,142.91,143.62,163.7,169.82,139.88,142.41,144.36,143.45,156.59,173.96,175.32,163.1,170.72,165.23,139.45,146.82,154.9,155.16],[
134.31,145.91,163.24,168.03,149.29,159.95,139.68,149.9,134.87,140.36,154.75,169.73,151.33,138.89,144.22,141.52,168.77,155.77,175.92,154.46,167.47,164.62,132.11,145.8,145.54,154.77],[
145.34,144.7,178.22,167.78,155.39,158.98,148.17,148.94,142.37,141.34,162.14,166.95,181.74,150.35,145.95,141.3,200.44,167.07,177.25,161.2,162.07,165.75,149.99,147.84,148.64,151.66],[
143.9,144.44,177.87,166.29,152.1,158.37,143.23,147.42,138.7,140.92,160.56,164.51,163.77,147.3,145.26,141.56,171.2,164.62,176.66,157.44,163.83,164.37,161.21,148.5,141.37,148.09],[
146.77,143.75,165.88,167.38,155.92,157.43,145.74,146.04,139.47,140.19,180.51,157.55,135.63,138.23,141.55,140.29,154.71,175.05,174.52,160.88,160.86,160.95,149,146.88,141.05,146.38],[
153.53,142.45,163.77,166.56,155.86,156.12,146.85,146.4,137.94,139.06,167.5,163.11,129.34,135.87,138.21,139.97,154.45,193.94,175.05,157.42,160.34,161.76,151.52,146.85,140.28,146.51],[
155.98,140.92,193.3,163.16,162.37,155.66,150.32,142.88,139.76,138.53,164.26,159.88,132.43,135.28,139.25,136.92,154.97,187.21,172.86,160.76,158.24,154.69,159.07,148.12,140.87,145],[
146.79,143.96,177.23,164.49,161.59,155.41,152.16,144.11,140.98,139.68,162.27,162.89,130.11,134.67,138.88,138.57,158.37,178.87,172.62,167.2,161.94,158.11,146.63,145.88,141.78,143.43],[
140.69,142.58,161.19,164.58,160.35,154.64,148.04,143.26,136.63,137.33,158.84,159.14,127.15,132.48,137.41,136.84,155.59,177.67,175.08,167.96,157.45,159.22,138.57,142.28,146.22,142.16],[
142.93,142.7,159.7,165.28,158.41,154.82,148.32,143.88,139.39,137.68,158.75,156.99,130.63,136.99,136.77,136.31,151.95,167.63,171.37,160.06,155.2,158.03,140.7,143.54,150.02,140.66],[
135.44,140.19,151.3,165.83,153.36,153.83,142.08,141.98,137.39,136.75,144.36,146.52,130.52,137.38,135.71,135.87,149.12,165.18,171.11,156.99,152.12,156.12,137.02,140.81,149.49,142.14],[
134.37,140.7,148.5,164.86,156.13,152.79,140.7,143.31,141.06,137.16,145.02,146.64,133.53,136.44,137.14,134.56,149.56,178.26,170.45,152.72,150.74,155.85,140.21,142.28,152.27,139.12],[
129.34,139.38,144.03,162.3,148.62,153.8,133.51,143.1,134.98,136.27,141.95,148.17,131.8,131.85,137.49,134.63,146.62,166.3,168.47,140.18,148.1,156.29,134.69,142.98,138.17,139.34],[
130.21,139.3,159.85,162.83,145.28,153.43,134.26,141.82,132.1,135.04,142.5,152.44,144.84,134.6,137.26,134.13,162.31,156.06,170.17,138.44,148.77,154.83,129.04,140,132.75,138.84],[
140.62,140.33,171.09,160.47,150.23,154.1,140.97,142.07,135.54,134.9,144.93,149.2,168.83,143.24,134.83,134.4,187.86,161.21,170.89,152.16,151.79,154.56,141.21,139,136.39,139.81],[
140.89,139.88,174.77,161.58,147.99,152.95,140.21,143.44,132.76,134.79,144.94,147.36,154.38,142.05,135.72,135.46,164.84,160.41,172.37,142.65,146.11,157.97,149.44,135.36,134.76,140.11],[
143.06,139.83,161.07,162.28,150.46,151.57,140.9,141,134.49,135.06,166.47,145.28,131.99,133.25,137.38,135.09,149.38,169.47,169.76,154.35,154.52,155.91,139.16,137.46,135.3,140.21],[
152.07,140.23,161.96,164.14,150.97,150.66,142.52,140.62,134.75,135.11,150.08,144.71,128.79,134.01,136.39,136.21,151.35,190.62,169.05,150.96,151.75,155.86,140.91,136.58,134.92,140.24],[
154.37,141.08,189.4,160.68,154.73,149.72,146.61,140.59,135.51,135.21,149.6,146.43,126.75,131.36,135.55,135.29,153.44,182.43,168.75,153.99,153.53,155.54,140.74,132.6,136.31,140.82],[
143.51,139.51,176.35,161.86,156.6,148.32,147.79,138.92,136.26,134.37,145.18,144.5,129.54,133.87,136.45,135.5,155.11,179.63,172.85,160.41,152.39,153.47,137.12,135.64,139.53,140.01],[
138.15,139.47,159.45,162.63,153.95,148.68,145,139.73,132.6,132.94,143.85,145.33,125.44,131.91,135.49,136.05,151.03,172.54,169,158.43,151.35,154.08,132.64,135.77,144.04,139.7],[
136.04,136.7,153.96,161.42,151.01,149.04,141.55,138.59,133.6,132.16,146.53,147.34,128.06,132.35,136.18,133.93,148.15,159.33,164.5,149.86,148.4,156.33,134.02,137.52,148.51,139.76],[
133.05,137.67,145.49,160.05,148.67,148.86,139.31,139.25,132.61,131.9,142.56,144.69,130.52,136.11,135.96,134.49,150.29,159.98,165.59,154.76,150.15,157.06,132.26,135.51,146.13,138.61],[
132.22,137.05,144.71,162.12,151.88,147.65,136.24,137.5,134.4,129.78,145.7,146.22,131.87,136.91,134.64,133.77,149.05,171.32,163.92,150.98,148.7,153.66,134.05,134.97,151.75,137.86],[
128.58,137.09,141.47,159.49,146.66,149.85,131.35,139.39,130.35,130.51,143.68,149.35,134.11,136.21,137.82,137.25,149.43,166.83,168.31,140.63,145.96,157.53,127.92,134.43,139.71,139.95],[
128.94,139.68,162.96,167.06,141.02,150.59,130.93,139.81,127.12,131.93,133.71,146.58,152.1,139.38,143.52,141.48,167.19,148.83,167.91,138.02,147.82,162.21,123.31,135.54,130.13,138.97],[
142.26,141.22,178.79,165.38,147.1,150.48,139.28,139.85,131.9,130.91,143.38,147.43,180.93,150.32,143.59,139.74,197.76,161.41,170.16,147.17,146.94,162.31,140.36,137.44,136.7,139.99],[
139.28,140.73,177.39,166.09,143.06,151.5,135.05,140.85,128.33,132.1,143,148.63,159.05,144.75,142.31,141.8,163.08,158.86,173.28,143.58,150.71,158.79,151.33,138.32,133.01,140.52],[
143.63,139.66,163.39,164.03,155.76,156.29,142.65,141.52,130.3,130.5,177.81,153.41,134.93,138.83,139.3,139.02,153.61,177.5,177.24,156.12,153.77,158.01,140.33,138.14,136.01,140.39],[
150.86,140.02,161.84,164.77,153,153.82,145.12,144.05,129.22,130.15,157.32,152.66,129.05,135.43,137.68,139.23,152.03,201.67,178.75,151.44,155.03,158.47,137.39,134.68,134.72,140.5],[
152.36,139.61,203.02,173.79,159.94,154.74,150.01,144,128.92,128.41,151.82,149.04,128.24,135.26,137.68,139.93,157.63,191.56,176.53,155.82,155.39,159.9,141.71,134.21,135.4,139.84],[
142.8,138.84,178.49,163.54,161.28,152.55,152.43,143.27,131,129.46,149.33,148.44,129.26,137.9,135.75,139.14,162.2,181.67,175.04,165.44,157.17,159.99,138.27,137.08,140.77,141.15],[
136.26,137.72,159.42,163.4,157.29,151.46,149.37,144.44,130.83,131.15,142.77,142.83,124.77,134.86,136.13,139.96,154.75,177.85,174.03,166.27,157.79,159.4,133.92,137.65,144.55,140.29],[
137.31,137.66,155.92,163.37,154.59,151.36,146.12,142.96,132.68,131.15,146.18,146.34,128.23,138.55,135.63,139.22,150.66,164.17,169.44,161.5,158.29,159.26,133.37,136.35,150.44,140.81],[
135,139.53,148.85,163.83,151.61,151.5,143.7,143.56,131.88,131.14,145.85,148.11,131.22,143.29,136.63,141.26,153.53,164.37,169.99,161.01,156.15,160.17,134.42,137.23,149.91,141.82],[
134.05,140.93,144.68,163.36,156,153.47,140.29,143.73,133.6,130.39,146.87,148.13,136.74,144.28,141.84,143.54,153.88,177.21,172.35,162.04,163.54,162.12,134.76,137.57,155.18,142.61],[
131.76,140.58,147.68,164.45,150.1,153.16,134.84,143.17,129.93,129.94,143.82,148.12,135.52,141.92,139.21,142.61,150.43,176.06,177.46,161.48,165.98,158.72,132.78,139.57,140.9,141.36],[
127.67,138.89,162.06,165.85,141.82,152.35,133.05,142.44,125.47,130.61,137.23,151.14,147.29,141.18,139.26,144.14,164.09,153.91,173.96,150.03,161.79,159.8,125.97,138.56,131.71,141.3],[
140.88,141.09,177.53,165.56,149.29,154.53,143.48,145.37,130.55,130.32,147.92,153.51,171.57,151.33,137.98,142.75,192.51,166.42,177.21,163.7,164.66,159.52,141.56,139.42,136.92,141.5],[
143,142.45,179.36,166.01,147.75,154.7,142.72,147.29,130.17,133.09,149.29,155.05,158.86,151.21,138.85,144.8,169.48,160.86,174.01,155.1,162.02,162.97,156.89,141.24,134.55,140.86],[
140.2,137.49,156.78,157.47,151.27,153.76,141.17,141.17,128.84,129.59,186.29,161.98,132.37,136.29,138.22,138.04,147.47,172.99,173.68,156.74,157.4,153.04,135.95,135.31,133.26,138.62],[
155.1,143.37,161.52,166.02,154.43,154.82,148.17,146.1,129.56,129.99,162.65,156.77,133.22,140.87,141.95,144.68,157.56,199.48,175.48,152.11,153.49,163.04,142.06,139.19,137.72,143.13],[
155.78,142.73,197.12,166.75,164.53,158.11,152.82,146.52,130.11,129.62,158.39,155.6,134.89,142.12,143.7,146.1,160.42,191.33,176.15,158.99,158.17,162.47,146.81,139.25,139.42,143.46],[
145.67,141.34,182.26,167.32,167.72,158.22,155.43,146.16,130.71,128.96,154.87,153.48,138.31,144.99,145.3,145.84,165.25,182.11,174.86,162.65,155.63,162.83,140.47,139.76,146.23,146.57],[
137.25,139.82,161.83,166.68,162.63,157.85,149.24,145.25,128.49,129.34,150.76,152.33,132.47,138.32,145.83,145.27,157.56,179.02,176.1,158.19,152.09,163.05,134.43,139.15,150.25,146.69],[
140.96,140.26,159.52,165.77,159.92,155.29,148.08,143.71,131.72,129.44,154.13,153.13,137.98,146.89,144.72,145.92,156.49,172.75,176.56,166.34,159.7,163.21,138.14,139.96,153.6,142.78],[
136.91,141.3,150.02,165.4,156.56,156.08,146.11,145.77,130.39,129.61,153.01,155.37,139.7,148.48,145.05,145.96,158.04,171.41,177.02,163.08,157.92,164.71,137.04,139.4,150.99,142.46],[
135.36,142.07,150.42,169.19,158.29,156.02,142.8,146.41,134.12,130.24,152.82,154.87,138.12,144.59,144.15,144.58,156.11,179.9,175,155.26,158.14,164.57,136.32,139.16,154.24,142.17],[
134.9,143.79,151.63,167.7,152.78,156.29,137.98,146.29,131.3,130.84,150.69,155.3,139.94,145.34,143.73,145.83,156.28,174.98,175.76,157.36,162.29,164.37,131.05,137.41,140.92,141.57],[
131.56,141.08,166.37,168.03,149.15,157.87,139.87,147.37,126.37,130.32,139.66,151.54,155.16,145.53,144.12,145.72,171.43,159.03,178.01,155.68,162.54,163.46,128.26,138.7,134.92,143.17],[
139.12,141.79,179.26,168.63,150.4,158.48,143.57,148.12,129.72,130.97,146.95,154.5,175.83,154.21,144.23,148.33,192.18,163.9,176.62,156.79,163.14,163.68,138.77,139.04,135.87,142.49],[
141.2,140.15,181.07,169.06,151.76,158.99,145.25,149.56,128.67,131.59,145.81,150.84,165.5,154.99,144.54,148.85,171.26,155.75,169.3,158.09,161.86,163.5,155.12,138.45,138.52,144.84],[
143.7,139.65,167.87,168.88,158.32,160.34,150.8,149.2,132.53,132.78,171.94,149.72,141.08,149.12,145.81,149.34,161.35,178.97,179.21,163.39,163.06,164.86,142.21,140.6,139.86,144.62],[
148.35,139.7,163.06,169.02,155.52,158.7,148.4,148.9,128.25,129.98,153.46,150.05,133.82,141.43,145.42,149.12,156.49,203.26,180.51,158.06,163.62,165.82,139.93,139.95,136.71,143.91],[
152.17,138.74,205.27,173.24,167.18,159.05,154.8,147.68,133.96,132.67,153.38,149.27,136.51,145.8,144.34,148.27,165.11,196.28,179.59,162.71,159.33,165.76,149.41,141.22,138.61,141.79],[
144.92,140.52,183.2,168.92,168.22,158.8,157.77,148.56,134.83,133.17,150.84,149.47,135.77,146.55,143.37,148.32,168.15,186.24,179.14,170.52,163.64,165.77,138.52,138.13,143.64,144.07],[
140.02,141.47,164.17,168.84,165.19,158.53,153.03,148,133.34,133.44,147.93,149.95,132.11,143.9,143.28,148.85,160.98,179.53,175.64,168.1,162.82,165.65,133.9,138.19,149.15,144.77],[
143.11,141.75,166.19,171.35,165.22,159.45,154.66,149.21,136.29,133.9,149.86,148.38,138.33,151.49,143.27,148.42,162.61,176.95,179.33,170.86,163.51,168.57,137.62,138.59,157.92,145.86],[
137.46,141.69,154.75,170.17,161.24,160.42,148.55,147.97,137.16,136.33,145.06,147.04,139.79,151.29,144.74,148.27,158.44,175.41,180.95,170.1,164.53,165.03,136.41,138.26,153.8,144.78],[
134,139.95,150.76,169.28,163.25,159.73,144.81,147.72,139.23,135.53,145.11,147.15,138.84,149.86,143.64,148.93,156.21,185.72,181.58,161.63,164.02,164.31,136.64,138.47,158.9,145.88],[
131.84,140.04,154.82,171.02,157.03,160.55,141.01,149.07,136.39,135.67,143.15,147.72,138.7,149.05,142.28,149.16,159.23,177.31,178.15,163.62,169.49,167.45,132.94,138.88,145.53,145.89],[
131.39,141.41,170.08,173.16,150.74,160.82,140.8,148.55,131.11,134.67,140.14,149.13,147.9,150.08,139.93,150.31,180.42,166.49,181.06,156.71,166.98,171.68,129.86,140.01,140.7,148.13],[
140.06,140.74,183.4,170.19,147.37,152.62,143.37,145.85,131.37,131.39,140.06,146.4,160.41,146.93,129.08,138.65,204.54,165.84,178.35,156.76,158.23,171.07,138.03,136.44,136.58,142.03],[
132.63,133.14,177.05,165.22,132.85,141.34,129.49,134.76,120.41,123.76,127.43,132.44,137.29,136.87,120.94,132.72,158.23,158.88,173,124.33,129.56,152.07,142.76,128.55,125.27,132.36],[
136.75,134.36,162,164.69,139.15,142.84,136.35,136.58,122.02,123.07,151.01,132.66,126.51,133.75,133.27,136.86,150.53,171.1,173.17,133.98,135.02,156.09,128.96,129.06,129.7,135.49],[
147.46,137.72,162.17,167.34,149.1,150.61,143.66,142.7,124.58,125.45,137.15,134.02,127.25,136.43,136.37,142.11,151.61,203.97,180.13,154.21,157.48,158.93,131.4,130.68,134.24,140.24],[
153.72,140.46,201.34,168.59,164.42,156.39,153.51,146.49,131.05,129.77,146.42,142.36,131.37,142.72,138.43,144.29,158.56,199.11,181.7,167.54,165.16,158.61,139.59,131.94,141.55,144.57],[
145.14,141.58,181.89,168.81,169.93,161.25,155.33,147.78,132.23,130.92,143.74,143.29,132.61,144.96,141.9,148.9,162.83,190.84,184.43,171.81,167.03,162.57,132.38,133.39,145.43,146.89],[
142.22,142.79,166.99,169.93,168.33,159.92,156.72,150.52,130.56,130.68,146.5,145.7,133.09,148.95,143.44,152.45,163.99,188.23,183.91,179.58,168.91,167.12,131.77,135.19,152.4,147.06],[
144.56,143.27,164.12,169.54,165.42,160.48,157.24,151.75,132.56,130.56,149.81,147.62,138.68,156.73,144.95,154.56,164.74,184.35,186.26,176.83,169.4,171.34,135.35,136.22,161.09,148.71],[
139.81,143.96,154.84,170.47,161.52,160.43,153.67,152.84,130.83,130.03,146.65,148.47,140.43,157.08,145.06,153.6,162.9,178.1,183.54,179.43,173.31,169.66,135.9,137.4,159.29,149.74],[
139.92,145.39,153.31,170.21,166.47,161.58,151.81,153.75,134.02,129.98,151.19,150.95,141.54,156.85,145.31,153.91,164.83,190.14,184.53,173.64,171.86,171.16,138.03,138.88,162.88,149.15],[
135.65,145.49,151.35,169.59,156.07,161.35,144.26,154.13,131.13,131.1,144.74,150.15,140,151.3,146.2,154.35,160.77,183.28,185.92,151.86,159.01,171.67,131.06,137.98,149.3,150.82],[
133.29,145.43,167.6,170.07,148.35,160.39,143.88,153.83,126.19,131.65,139.08,152.58,160.78,150.88,151.53,154.12,176.41,164.3,184.77,148.43,159.98,171.18,124.87,136.74,141.23,151.41],[
146.26,146.31,182.79,171.24,148.71,153.02,151.67,153.46,130.74,130.25,147.65,152.93,188.83,163.89,151.16,153.25,205.89,173.01,185.44,169.28,168.28,170.9,139.2,137.06,143.13,148.58],[
146.98,147.3,183.98,173.03,150.35,160.21,147.83,153.64,128.66,132.61,148.13,155.07,173.78,159.75,152.39,155.19,177.57,170.49,186.83,164.82,172.7,170.62,155.38,139.46,142.11,149.81],[
151.24,148.21,173.05,175.66,156.61,160.53,154.38,154.48,132.9,133.65,175.56,155.42,140.61,153.77,147.55,156.89,164.29,184.96,187.08,169.69,171.59,169.95,137.67,137.94,146.03,152.59],[
155.87,146.07,172.09,176.64,159.82,161.46,156.22,155.3,132.97,134.09,159.28,154.91,134.82,151.59,144.43,157.59,160.21,211.88,186.92,172.5,175.01,167.9,137.98,137.63,146.84,153.54],[
157.74,144.48,209.21,176.81,169.41,161.65,161.2,154.15,135.79,134.64,159.66,155.74,137.09,156.04,145.3,158.87,169.07,203.23,186.21,167.9,166.23,169.46,145.73,137.72,150.26,153.4]],columns=IBCR, index=[meses])
urllib.request.urlretrieve('https://www.bcb.gov.br/content/estabilidadefinanceira/certificados-digitais/br.gov.bcb.wildcard-full-chain-2020.crt.zip','downloads/czip.zip')
#unzip o certificado
with ZipFile('downloads/czip.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall('downloads/')
#usando o certificado
wsdl = 'https://www3.bcb.gov.br/sgspub/JSP/sgsgeral/FachadaWSSGS.wsdl'
session = Session()
session.verify = 'downloads/br.gov.bcb.wildcard-full-chain.crt'
transport = Transport(session=session)
client = Client(wsdl, transport=transport)
mesesload = pd.to_datetime(pd.date_range('2021-08-01',datetime.today(),freq = '1MS')).strftime('%d/%m/%Y')
df = pd.DataFrame(columns=IBCR, index=[mesesload])
for codigo in IBCR:
  listcode = []
  for date in mesesload:
    try:
      codres = client.service.getValor(codigo,date)
      codres = str(codres)
      s = [float(s) for s in re.findall(r'-?\d+\.?\d*', codres)]
      s = s[1]
    except:
      s = None
    listcode.append(s)
    if len(mesesload) == len(listcode):
       df[codigo] = listcode 
df.dropna(how = 'all', inplace = True)
final = preload.append(df)
IBCRDESCRITIVO = {'25379': '25379 - Índice de Atividade Econômica Regional - Minas Gerais - Índice',
 '25380': '25380 - Índice de Atividade Econômica Regional - Minas Gerais - com ajuste sazonal - Índice',
 '25383': '25383 - Índice de Atividade Econômica Regional - Goiás - Índice',
 '25384': '25384 - Índice de Atividade Econômica Regional - Goiás - com ajuste sazonal - Índice',
 '25390': '25390 - Índice de Atividade Econômica Regional - Ceará - Índice',
 '25391': '25391 - Índice de Atividade Econômica Regional - Ceará - com ajuste sazonal - Índice',
 '25392': '25392 - Índice de Atividade Econômica Regional - São Paulo - Índice',
 '25394': '25394 - Índice de Atividade Econômica Regional - São Paulo - com ajuste sazonal - Índice',
 '25396': '25396 - Índice de Atividade Econômica Regional - Rio de Janeiro - Índice',
 '25397': '25397 - Índice de Atividade Econômica Regional - Rio de Janeiro - com ajuste sazonal - Índice',
 '25398': '25398 - Índice de Atividade Econômica Regional - Espírito Santo - Índice',
 '25399': '25399 - Índice de Atividade Econômica Regional - Espírito Santo - com ajuste sazonal - Índice',
 '25401': '25401 - Índice de Atividade Econômica Regional - Rio Grande do Sul - Índice',
 '25402': '25402 - Índice de Atividade Econômica Regional - Santa Catarina - Índice',
 '25404': '25404 - Índice de Atividade Econômica Regional - Rio Grande do Sul - com ajuste sazonal - Índice',
 '25405': '25405 - Índice de Atividade Econômica Regional - Santa Catarina - com ajuste sazonal - Índice',
 '25408': '25408 - Índice de Atividade Econômica Regional - Paraná - Índice',
 '25409': '25409 - Índice de Atividade Econômica Regional - Pará - Índice',
 '25410': '25410 - Índice de Atividade Econômica Regional - Pará - com ajuste sazonal - Índice',
 '25411': '25411 - Índice de Atividade Econômica Regional - Amazonas - Índice',
 '25412': '25412 - Índice de Atividade Econômica Regional - Amazonas - com ajuste sazonal - Índice',
 '25413': '25413 - Índice de Atividade Econômica Regional - Paraná - com ajuste sazonal - Índice',
 '25415': '25415 - Índice de Atividade Econômica Regional - Bahia - Índice',
 '25416': '25416 - Índice de Atividade Econômica Regional - Bahia - com ajuste sazonal - Índice',
 '25417': '25417 - Índice de Atividade Econômica Regional - Pernambuco - Índice',
 '25418': '25418 - Índice de Atividade Econômica Regional - Pernambuco - com ajuste sazonal - Índice'}

final.rename(columns = IBCRDESCRITIVO, inplace= True)
final.at['01/01/2003', 'Fonte - dados'] = 'https://www.bcb.gov.br/ - Índice de Atividade Econômica Regional' 
final.to_excel('raws/BCB - IBCR.xlsx')



In [12]:
from os import listdir
from os.path import isfile, join
datalist = [f for f in listdir('raws/') if isfile(join('raws/', f))]
datalist = sorted(datalist)

path = 'data.xlsx'
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(path, engine='xlsxwriter')

for caminho in datalist:
  stringu = 'raws/'+ caminho
  df = pd.read_excel(stringu)
  # Convert the dataframe to an XlsxWriter Excel object.
  sheet_name =  caminho.replace(".xlsx", "")
  df.to_excel(writer, sheet_name = sheet_name)


#aqui fechar o for loop
# Close the Pandas Excel writer and output the Excel file.
writer.save()